In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
df = pd.read_csv('深圳写字楼数据.csv')

In [3]:
def replacestr(string):
    string = str(string).replace('[','')
    string = string.replace(']','')
    string = string.replace("'",'')
    return string

df['loupandizhi'] = df['loupandizhi'].apply(replacestr)

list_location = df['loupandizhi'].value_counts().index.tolist()

dict_location = {}
Key = '6b95b84cab6100c0a6f52c62ece97e4b'
My_key = 'f654d644aeae0e1da847d7ca408f4370'
url_pre = 'https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}'
counter = 0
for i in list_location:
    location = i.split(",")[-1].strip()
    url = url_pre.format(Key, location, '深圳')
    res = requests.get(url).json()
    if res['status'] == '0':
        dict_location[i] = np.nan
        counter +=1
    else:
        try: 
            dict_location[i] = res['geocodes'][0]['location']
            if counter % 250 == 0:
                print('one batch(250) processed')
        except:
            dict_location[i] = np.nan
            
        counter +=1

one batch(250) processed
one batch(250) processed


In [4]:
df['location_coords'] = np.nan

counter = 0
for i in list_location:
    counter += 1
    df_temp = df[df['loupandizhi'] == i]
    df_temp_index = df_temp.index.tolist()
    df.iloc[df_temp_index,14] = dict_location[i]
    if counter %50 == 0:
        print('A batch(50) is done')

A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done
A batch(50) is done


In [5]:
df.head(3)

,building_type,dengji,jianzhumianji,loupan_content,loupandizhi,price_perunit,release_date,service_price,suozailouceng,titile,total_price_10K,url,xzl_content,zhuangxiu,location_coords
0,纯写字楼,甲级,2200㎡,NaN,"深圳, 罗湖, 布心, IBC环球商务中心",(20.45元/㎡),2019/9/6 12:33:52,23.00元/平米·月,中区(共35层),罗湖 环球商务中心 一手开发商275–1800平 户型方正实用珠宝齐聚,4,https://sz.office.fang.com/shou/3_221494519.html,房源亮点【可售楼层】高层仅此一层内部放售！现楼！超钾级！【可选面积】整层等多种面积可选【项目...,简装修(可分割),"114.124864,22.574536"
1,纯写字楼,甲级,2032㎡,深圳荣超经贸中心位于深圳中心商务区（C B D ）北区，毗邻深圳市民中心，总高度达200m ...,"深圳, 福田, 福田中心区, 荣超经贸中心",(56000元/㎡),2019/9/6 11:55:06,16.00元/平米·月,中区(共48层),福田中心区，市民中心旁地标物业，整层单位，精装出售，各付各税,11379,https://sz.office.fang.com/shou/3_221492499.html,物业地址：深圳市福田区金田路与福中路交界东南物业管理：深圳市荣超物业有限公司开发 商：深圳市...,精装修(可分割),"114.063658,22.544042"
2,纯写字楼,甲级,2000㎡,万科大厦高约155米，是龙岗核心地段的商务写字楼。“天际office”为万科大厦特别推出的3...,"深圳, 龙岗, 龙岗中心城, 万科大厦",(36000元/㎡),2019/9/8 13:46:35,14.24元/平米·月,低区(共32层),龙岗中心城 万科大厦 整层打包出售 带豪装 租20万/月,7200,https://sz.office.fang.com/shou/3_221558546.html,万科大厦高约155米，是龙岗的高度，是龙岗的商务地标。“天际office”是万科大厦推出的3...,精装修(不可分割),"114.242657,22.713481"


In [6]:
df.to_csv('深圳写字楼地址数据.csv', encoding = 'utf-8-sig',index = False)